In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Krok 1: Definujte Pydantic modely pro strukturované výstupy

Tyto modely definují **schéma**, které agenti budou vracet. Použití `response_format` s Pydantic zajišťuje:
- ✅ Typově bezpečnou extrakci dat
- ✅ Automatickou validaci
- ✅ Žádné chyby při analýze volného textu
- ✅ Snadné podmíněné směrování na základě polí


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Krok 2: Vytvoření nástroje pro rezervaci hotelu

Tento nástroj bude volán **availability_agent**, aby ověřil dostupnost pokojů. Používáme dekorátor `@ai_function` k:
- Převodu funkce v Pythonu na nástroj, který může volat AI
- Automatickému generování JSON schématu pro LLM
- Ověření parametrů
- Umožnění automatického vyvolání agenty

Pro tento demo:
- **Stockholm, Seattle, Tokio, Londýn, Amsterdam** → Pokoje jsou dostupné ✅
- **Všechna ostatní města** → Pokoje nejsou dostupné ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Krok 3: Definujte funkce podmínek pro směrování

Tyto funkce kontrolují odpověď agenta a určují, kterou cestou se workflow vydá.

**Klíčový postup:**
1. Zkontrolujte, zda je zpráva `AgentExecutorResponse`
2. Analyzujte strukturovaný výstup (Pydantic model)
3. Vraťte `True` nebo `False` pro řízení směrování

Workflow vyhodnotí tyto podmínky na **hranách**, aby rozhodl, který executor bude následně spuštěn.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Krok 4: Vytvoření vlastního vykonavatele zobrazení

Vykonavatelé jsou komponenty pracovního postupu, které provádějí transformace nebo vedlejší efekty. Používáme dekorátor `@executor` k vytvoření vlastního vykonavatele, který zobrazí konečný výsledek.

**Klíčové koncepty:**
- `@executor(id="...")` - Registruje funkci jako vykonavatele pracovního postupu
- `WorkflowContext[Never, str]` - Typové nápovědy pro vstup/výstup
- `ctx.yield_output(...)` - Vydává konečný výsledek pracovního postupu


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Krok 5: Načtení proměnných prostředí

Nakonfigurujte klienta LLM. Tento příklad funguje s:
- **GitHub Models** (bezplatná verze s tokenem GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Krok 6: Vytvoření AI agentů se strukturovanými výstupy

Vytvoříme **tři specializované agenty**, každý zabalený v `AgentExecutor`:

1. **availability_agent** - Kontroluje dostupnost hotelů pomocí nástroje
2. **alternative_agent** - Navrhuje alternativní města (pokud nejsou dostupné pokoje)
3. **booking_agent** - Povzbuzuje k rezervaci (pokud jsou pokoje dostupné)

**Klíčové vlastnosti:**
- `tools=[hotel_booking]` - Poskytuje agentovi nástroj
- `response_format=PydanticModel` - Vynucuje strukturovaný výstup ve formátu JSON
- `AgentExecutor(..., id="...")` - Zabalí agenta pro použití ve workflow


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Krok 7: Vytvoření pracovního postupu s podmíněnými hranami

Nyní použijeme `WorkflowBuilder` k vytvoření grafu s podmíněným směrováním:

**Struktura pracovního postupu:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Klíčové metody:**
- `.set_start_executor(...)` - Nastaví vstupní bod
- `.add_edge(from, to, condition=...)` - Přidá podmíněnou hranu
- `.build()` - Finalizuje pracovní postup


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Krok 8: Spusťte testovací případ 1 - Město BEZ dostupnosti (Paříž)

Otestujme cestu **bez dostupnosti** tím, že požádáme o hotely v Paříži (která v naší simulaci nemá žádné pokoje).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Krok 9: Spuštění testovacího případu 2 - Město S dostupností (Stockholm)

Nyní otestujeme cestu **dostupnosti** tím, že požádáme o hotely ve Stockholmu (který má pokoje v naší simulaci).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Klíčové poznatky a další kroky

### ✅ Co jste se naučili:

1. **Vzorec WorkflowBuilder**
   - Použijte `.set_start_executor()` k definování vstupního bodu
   - Použijte `.add_edge(from, to, condition=...)` pro podmíněné směrování
   - Zavolejte `.build()` k dokončení workflow

2. **Podmíněné směrování**
   - Funkce podmínek zkoumají `AgentExecutorResponse`
   - Analyzujte strukturované výstupy pro rozhodování o směrování
   - Vrácení hodnoty `True` aktivuje hranu, `False` ji přeskočí

3. **Integrace nástrojů**
   - Použijte `@ai_function` k převodu funkcí Pythonu na AI nástroje
   - Agenti volají nástroje automaticky, když je to potřeba
   - Nástroje vrací JSON, který mohou agenti analyzovat

4. **Strukturované výstupy**
   - Použijte Pydantic modely pro typově bezpečnou extrakci dat
   - Nastavte `response_format=MyModel` při vytváření agentů
   - Analyzujte odpovědi pomocí `Model.model_validate_json()`

5. **Vlastní exekutory**
   - Použijte `@executor(id="...")` k vytvoření komponent workflow
   - Exekutory mohou transformovat data nebo provádět vedlejší efekty
   - Použijte `ctx.yield_output()` k produkci výsledků workflow

### 🚀 Reálné aplikace:

- **Rezervace cestování**: Kontrola dostupnosti, návrh alternativ, porovnání možností
- **Zákaznický servis**: Směrování podle typu problému, sentimentu, priority
- **E-commerce**: Kontrola skladových zásob, návrh alternativ, zpracování objednávek
- **Moderace obsahu**: Směrování podle skóre toxicity, uživatelských hlášení
- **Schvalovací workflow**: Směrování podle částky, role uživatele, úrovně rizika
- **Vícefázové zpracování**: Směrování podle kvality dat, úplnosti

### 📚 Další kroky:

- Přidejte složitější podmínky (více kritérií)
- Implementujte smyčky s řízením stavu workflow
- Přidejte pod-workflow pro opakovaně použitelné komponenty
- Integrujte s reálnými API (rezervace hotelů, skladové systémy)
- Přidejte zpracování chyb a záložní cesty
- Vizualizujte workflow pomocí vestavěných nástrojů pro vizualizaci



---

**Upozornění**:  
Tento dokument byl přeložen pomocí služby AI pro překlad [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o přesnost, mějte prosím na paměti, že automatizované překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace se doporučuje profesionální lidský překlad. Nejsme zodpovědní za jakékoli nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
